In [1]:
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
url = 'https://storage.googleapis.com/taxi-data-de/taxi_data_0123.parquet'
df = pd.read_parquet(url)

look_up_url = 'https://storage.googleapis.com/taxi-data-de/taxi%2B_zone_lookup.csv'
df_look_up = pd.read_csv(look_up_url)

In [3]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [4]:
df_look_up

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [5]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [6]:
df = df.drop_duplicates().reset_index(drop=True)
df['Id'] = df.index

In [7]:
'''look_up_null_musk = df_look_up.isna().any(axis=1)
df_look_up_null = df_look_up[look_up_null_musk]
df_look_up_null'''

'look_up_null_musk = df_look_up.isna().any(axis=1)\ndf_look_up_null = df_look_up[look_up_null_musk]\ndf_look_up_null'

In [8]:
merged_df = df.merge(df_look_up.add_prefix('PU_'), left_on='PULocationID', right_on='PU_LocationID', how='left')
merged_df.drop('PU_LocationID', axis = 1, inplace = True)
merged_df = merged_df.merge(df_look_up.add_prefix('DO_'), left_on='DOLocationID', right_on='DO_LocationID', how='left')
merged_df.drop('DO_LocationID', axis = 1, inplace = True)

In [9]:
merged_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,total_amount,congestion_surcharge,airport_fee,Id,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,14.30,2.5,0.00,0,Manhattan,Midtown Center,Yellow Zone,Manhattan,Lenox Hill West,Yellow Zone
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,...,16.90,2.5,0.00,1,Manhattan,Central Park,Yellow Zone,Manhattan,Upper East Side South,Yellow Zone
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,...,34.90,2.5,0.00,2,Manhattan,Clinton East,Yellow Zone,Manhattan,Upper West Side North,Yellow Zone
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,...,20.85,0.0,1.25,3,Queens,LaGuardia Airport,Airports,Queens,Astoria,Boro Zone
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,...,19.68,2.5,0.00,4,Manhattan,Gramercy,Yellow Zone,Manhattan,East Village,Yellow Zone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,...,23.76,NaN,NaN,3066761,Manhattan,Gramercy,Yellow Zone,Manhattan,Clinton East,Yellow Zone
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,...,29.07,NaN,NaN,3066762,Brooklyn,Greenpoint,Boro Zone,Manhattan,East Harlem South,Boro Zone
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,...,26.93,NaN,NaN,3066763,Manhattan,Greenwich Village South,Yellow Zone,Manhattan,Upper West Side South,Yellow Zone
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,...,26.58,NaN,NaN,3066764,Manhattan,Times Sq/Theatre District,Yellow Zone,Manhattan,East Village,Yellow Zone


In [10]:
df = merged_df

## Datetime dimension table

In [11]:
Datetime_table = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)
Datetime_table['tpep_pickup_datetime'] = Datetime_table['tpep_pickup_datetime']
Datetime_table['pick_hour'] = Datetime_table['tpep_pickup_datetime'].dt.hour
Datetime_table['pick_day'] = Datetime_table['tpep_pickup_datetime'].dt.day
Datetime_table['pick_month'] = Datetime_table['tpep_pickup_datetime'].dt.month
Datetime_table['pick_year'] = Datetime_table['tpep_pickup_datetime'].dt.year
Datetime_table['pick_weekday'] = Datetime_table['tpep_pickup_datetime'].dt.weekday

In [12]:
Datetime_table['drop_hour'] = Datetime_table['tpep_dropoff_datetime'].dt.hour
Datetime_table['drop_day'] = Datetime_table['tpep_dropoff_datetime'].dt.day
Datetime_table['drop_month'] = Datetime_table['tpep_dropoff_datetime'].dt.month
Datetime_table['drop_year'] = Datetime_table['tpep_dropoff_datetime'].dt.year
Datetime_table['drop_weekday'] = Datetime_table['tpep_dropoff_datetime'].dt.weekday

In [13]:
Datetime_table['datetime_id'] = Datetime_table.index

In [14]:
Datetime_table = Datetime_table[['datetime_id',
                             'tpep_pickup_datetime',
                             'pick_hour',
                             'pick_day',
                             'pick_month',
                             'pick_year',
                             'pick_weekday',
                             'tpep_dropoff_datetime',
                             'drop_hour', 'drop_day',
                             'drop_month', 'drop_year',
                             'drop_weekday']]
#
Datetime_table.head()

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2023-01-01 00:32:10,0,1,1,2023,6,2023-01-01 00:40:36,0,1,1,2023,6
1,1,2023-01-01 00:55:08,0,1,1,2023,6,2023-01-01 01:01:27,1,1,1,2023,6
2,2,2023-01-01 00:25:04,0,1,1,2023,6,2023-01-01 00:37:49,0,1,1,2023,6
3,3,2023-01-01 00:03:48,0,1,1,2023,6,2023-01-01 00:13:25,0,1,1,2023,6
4,4,2023-01-01 00:10:29,0,1,1,2023,6,2023-01-01 00:21:19,0,1,1,2023,6


## Passenger table

In [15]:
passenger_table = df[['passenger_count']].reset_index(drop=True)
passenger_table['passenger_count_id'] = passenger_table.index
passenger_table = passenger_table[['passenger_count_id','passenger_count']]

In [16]:
passenger_table

,passenger_count_id,passenger_count
0,0,1.0
1,1,1.0
2,2,1.0
3,3,0.0
4,4,1.0
...,...,...
3066761,3066761,NaN
3066762,3066762,NaN
3066763,3066763,NaN
3066764,3066764,NaN


## Trip information table

In [17]:
trip_info = df[['trip_distance']].reset_index(drop=True)
trip_info['trip_distance_id'] = trip_info.index
trip_info = trip_info[['trip_distance_id','trip_distance']]

In [18]:
trip_info

,trip_distance_id,trip_distance
0,0,0.97
1,1,1.10
2,2,2.51
3,3,1.90
4,4,1.43
...,...,...
3066761,3066761,3.05
3066762,3066762,5.80
3066763,3066763,4.67
3066764,3066764,3.15


## Rate code table

In [19]:
rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code = df[['RatecodeID']].reset_index(drop=True)
rate_code['rate_code_id'] = rate_code.index
rate_code['rate_code_name'] = rate_code['RatecodeID'].map(rate_code_type)
rate_code = rate_code[['rate_code_id','RatecodeID','rate_code_name']]

In [20]:
rate_code

,rate_code_id,RatecodeID,rate_code_name
0,0,1.0,Standard rate
1,1,1.0,Standard rate
2,2,1.0,Standard rate
3,3,1.0,Standard rate
4,4,1.0,Standard rate
...,...,...,...
3066761,3066761,NaN,NaN
3066762,3066762,NaN,NaN
3066763,3066763,NaN,NaN
3066764,3066764,NaN,NaN


## Pickup location table

In [21]:
pickup_location = df[['PULocationID','PU_Borough','PU_Zone','PU_service_zone']].reset_index(drop=True)
pickup_location['pickup_location_id'] = pickup_location.index
pickup_location = pickup_location[['pickup_location_id','PULocationID','PU_Borough','PU_Zone','PU_service_zone']]

In [22]:
pickup_location

,pickup_location_id,PULocationID,PU_Borough,PU_Zone,PU_service_zone
0,0,161,Manhattan,Midtown Center,Yellow Zone
1,1,43,Manhattan,Central Park,Yellow Zone
2,2,48,Manhattan,Clinton East,Yellow Zone
3,3,138,Queens,LaGuardia Airport,Airports
4,4,107,Manhattan,Gramercy,Yellow Zone
...,...,...,...,...,...
3066761,3066761,107,Manhattan,Gramercy,Yellow Zone
3066762,3066762,112,Brooklyn,Greenpoint,Boro Zone
3066763,3066763,114,Manhattan,Greenwich Village South,Yellow Zone
3066764,3066764,230,Manhattan,Times Sq/Theatre District,Yellow Zone


## Dropoff location table

In [23]:
dropoff_location = df[['DOLocationID','DO_Borough','DO_Zone','DO_service_zone']].reset_index(drop=True)
dropoff_location['dropoff_location_id'] = dropoff_location.index
dropoff_location = dropoff_location[['dropoff_location_id','DOLocationID','DO_Borough','DO_Zone','DO_service_zone']]

In [24]:
dropoff_location

,dropoff_location_id,DOLocationID,DO_Borough,DO_Zone,DO_service_zone
0,0,141,Manhattan,Lenox Hill West,Yellow Zone
1,1,237,Manhattan,Upper East Side South,Yellow Zone
2,2,238,Manhattan,Upper West Side North,Yellow Zone
3,3,7,Queens,Astoria,Boro Zone
4,4,79,Manhattan,East Village,Yellow Zone
...,...,...,...,...,...
3066761,3066761,48,Manhattan,Clinton East,Yellow Zone
3066762,3066762,75,Manhattan,East Harlem South,Boro Zone
3066763,3066763,239,Manhattan,Upper West Side South,Yellow Zone
3066764,3066764,79,Manhattan,East Village,Yellow Zone


## Payment method table

In [25]:
payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_type_name)
payment_type_dim = payment_type_dim[['payment_type_id','payment_type','payment_type_name']]

In [26]:
payment_type_dim

,payment_type_id,payment_type,payment_type_name
0,0,2,Cash
1,1,1,Credit card
2,2,1,Credit card
3,3,1,Credit card
4,4,1,Credit card
...,...,...,...
3066761,3066761,0,NaN
3066762,3066762,0,NaN
3066763,3066763,0,NaN
3066764,3066764,0,NaN


## Fact table

In [28]:
fact_table = df.merge(passenger_table, left_on='Id', right_on='passenger_count_id') \
             .merge(trip_info, left_on='Id', right_on='trip_distance_id') \
             .merge(rate_code, left_on='Id', right_on='rate_code_id') \
             .merge(pickup_location, left_on='Id', right_on='pickup_location_id') \
             .merge(dropoff_location, left_on='Id', right_on='dropoff_location_id')\
             .merge(Datetime_table, left_on='Id', right_on='datetime_id') \
             .merge(payment_type_dim, left_on='Id', right_on='payment_type_id') \
             [['Id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',
               'pickup_location_id','dropoff_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [29]:
fact_table

,Id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,2,0,0,0,0,N,0,0,0,9.30,1.00,0.5,0.00,0.0,1.0,14.30
1,1,2,1,1,1,1,N,1,1,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90
2,2,2,2,2,2,2,N,2,2,2,14.90,1.00,0.5,15.00,0.0,1.0,34.90
3,3,1,3,3,3,3,N,3,3,3,12.10,7.25,0.5,0.00,0.0,1.0,20.85
4,4,2,4,4,4,4,N,4,4,4,11.40,1.00,0.5,3.28,0.0,1.0,19.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,3066761,2,3066761,3066761,3066761,3066761,None,3066761,3066761,3066761,15.80,0.00,0.5,3.96,0.0,1.0,23.76
3066762,3066762,2,3066762,3066762,3066762,3066762,None,3066762,3066762,3066762,22.43,0.00,0.5,2.64,0.0,1.0,29.07
3066763,3066763,2,3066763,3066763,3066763,3066763,None,3066763,3066763,3066763,17.61,0.00,0.5,5.32,0.0,1.0,26.93
3066764,3066764,2,3066764,3066764,3066764,3066764,None,3066764,3066764,3066764,18.15,0.00,0.5,4.43,0.0,1.0,26.58


# Load Dataframe into Bigquery

In [30]:
key_path = 'taxi-data-de-project-security-key.json'
credentials = service_account.Credentials.from_service_account_file(key_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [31]:
table_list = ['fact_table',
              'passenger_table',
              'trip_info',
              'rate_code',
              'pickup_location',
              'dropoff_location',
              'Datetime_table',
              'payment_type_dim']

dataframes = [fact_table,
             passenger_table,
             trip_info,
             rate_code,
             pickup_location,
             dropoff_location,
             Datetime_table,
             payment_type_dim]

# Change job config to write only (replace) 
job_config_replace = bigquery.job.LoadJobConfig()
job_config_replace.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

for table_name, dataframe in zip(table_list, dataframes):
    # Replace "dataset_name" with the name of your dataset in BigQuery
    table_id = 'taxi-data-de-project-389214.taxi_data_de.{}'.format(table_name)
    
    # Load the dataframe into BigQuery
    job = client.load_table_from_dataframe(dataframe, table_id, job_config=job_config_replace)
    
    # Wait for the job to complete
    job.result()
    
    # Print a success message
    print(f"Table {table_name} loaded successfully.")

Table fact_table loaded successfully.
Table passenger_table loaded successfully.
Table trip_info loaded successfully.
Table rate_code loaded successfully.
Table pickup_location loaded successfully.
Table dropoff_location loaded successfully.
Table Datetime_table loaded successfully.
Table payment_type_dim loaded successfully.
